In [ ]:
### Installation info
# conda create -n detectron_env python=3.8
# conda activate detectron_env
# conda install cython
# conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
# git clone https://github.com/facebookresearch/detectron2.git
# cd detectron2
# conda install -e .
# conda install opencv-python

In [ ]:
### Virtual environment and notebook init
# Call conda activate detectron_env
# cd C:\R_projects\deep_learning_playground
# Call jupyter lab

In [ ]:
### Check working directory
# pwd
### Check if entered correct environment
# conda info --envs

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import ColorMode,GenericMask,Visualizer
from detectron2.structures.keypoints import heatmaps_to_keypoints
from detectron2.structures.keypoints_hack import heatmaps_to_keypoints2, heatmaps_to_keypoints_iterative, heatmaps_modify_iterative
from matplotlib import pyplot as plt
import glob
import random
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    DatasetEvaluator,
    inference_on_dataset,
    print_csv_format,
    verify_results,
)
import cv2 

In [ ]:
cd C:\R_projects\deep_learning_playground\detectron2\custom_data2

In [ ]:
# Split training and test data
#!coco-split --has_annotations --valid_ratio 0 --test_ratio 0.1 --annotations_file ./caterpillar_12_09_2023_with_keypoints.json

In [ ]:
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
import torch
import time
import datetime
import logging

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

from detectron2.data import DatasetMapper, build_detection_test_loader
from detectron2.engine import DefaultTrainer

class CustomTrainer(DefaultTrainer):
    """
    Custom Trainer deriving from the "DefaultTrainer"

    Overloads build_hooks to add a hook to calculate loss on the test set during training.
    """

    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1, LossEvalHook(
            200, # Frequency of calculation - every 100 iterations here
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg, True)
            )
        ))

        return hooks


In [ ]:
register_coco_instances("cat_data_train", {}, ".\coco_train.json", ".\datasets\caterpillar")
register_coco_instances("cat_data_test", {}, ".\coco_test.json", ".\datasets\caterpillar")
#register_coco_instances("cat_data_val", {}, ".\valid.json", ".\datasets\caterpillar")
MetadataCatalog.get("cat_data_train").set(thing_classes=["cat"])
MetadataCatalog.get("cat_data_train").set(thing_colors=(0,0,255))
MetadataCatalog.get("cat_data_test").set(thing_classes=["cat"])
MetadataCatalog.get("cat_data_test").set(thing_colors=(0,0,255))
MetadataCatalog.get("cat_data_train").set(keypoint_names=["head","middle","tail"])
MetadataCatalog.get("cat_data_train").set(keypoint_flip_map=[])
MetadataCatalog.get("cat_data_test").set(keypoint_names=["head","middle","tail"])
MetadataCatalog.get("cat_data_test").set(keypoint_flip_map=[])

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("cat_data_train",)
cfg.DATASETS.TEST = ("cat_data_test", )
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.DATALOADER.NUM_WORKERS = 2
cfg.TEST.EVAL_PERIOD = 200
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00015  # pick a good LR
cfg.SOLVER.MAX_ITER = 3000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9
cfg.TEST.DETECTIONS_PER_IMAGE = 1
cfg.SOLVER.CHECKPOINT_PERIOD = 2000
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 3
cfg.TEST.KEYPOINT_OKS_SIGMAS = [0.1, 0.3, 0.1]
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.MODEL.MASK_ON = True
cfg.MODEL.KEYPOINT_ON = True
cfg.MODEL.ROI_KEYPOINT_HEAD.LOSS_WEIGHT = 5
cfg.MODEL.ROI_KEYPOINT_HEAD.NORMALIZE_LOSS_BY_VISIBLE_KEYPOINTS = False
#cfg.MODEL.ROI_KEYPOINT_HEAD.ITERATIVE = True
cfg.OUTPUT_DIR = "./output4"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


In [ ]:
print(cfg)

In [ ]:
#trainer = DefaultTrainer(cfg)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
%load_ext tensorboard
#%reload_ext tensorboard
%tensorboard --logdir "./output4"
# !del /S C:\Users\vsbpa\AppData\Local\Temp\.tensorboard-info

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_0001999.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9  # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
%matplotlib inline 

# f = glob.glob("C:\R_projects\deep_learning_playground\detectron2\custom_data\coco_dataset\*.jpg")
f = glob.glob("C:/R_projects/spat_1f_noise/processed_feed/rep50/*.jpg")
# f = ["C:/R_projects/spat_1f_noise/processed_feed/repweek2test2/processed_repweek2test2__cam28_s36764_rank103.jpg"]
fi = random.choice(f)
#fi = f[0]
im = cv2.imread(fi)
outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("cat_data_train"), 
                   scale=1, 
               instance_mode=ColorMode.IMAGE_BW
)
#out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
try: out = v.draw_circle((outputs["instances"].pred_keypoints.tolist())[0][0][:2:], "red", radius = 5)
except: print("Head not detected") 
try: out = v.draw_circle((outputs["instances"].pred_keypoints.tolist())[0][1][:2:], "black")
except: print("Middle not detected") 
try: out = v.draw_circle((outputs["instances"].pred_keypoints.tolist())[0][2][:2:], "green")
except: print("Tail not detected") 
try: out = v.draw_soft_mask(np.asarray(outputs["instances"].to("cpu").pred_masks)[0,:,:], "blue",alpha = 0.3)
except: print("Mask not detected") 

plt.figure(figsize = (10,10))
plt.imshow(out.get_image(), interpolation='nearest')
plt.title(fi)
plt.show()

In [ ]:
f = glob.glob("C:/R_projects/spat_1f_noise/processed_feed/rep50/*.jpg")
tot = len(f)
for i in range(tot):
    fi = f[i]
    im = cv2.imread(fi)
    outputs = predictor(im) 
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("cat_data_train"), 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW
    )
    #out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    try: out = v.draw_box(outputs["instances"].pred_boxes.tensor.tolist()[0])
    except: out = v.draw_circle((0,0), radius = 0, color = "black")
    try: out = v.draw_circle((outputs["instances"].pred_keypoints.tolist())[0][0][:2:], color = "red", radius = 3)
    except: 1
    try: out = v.draw_circle((outputs["instances"].pred_keypoints.tolist())[0][1][:2:], color = "black", radius = 2)
    except: 1
    try: out = v.draw_circle((outputs["instances"].pred_keypoints.tolist())[0][2][:2:], color = "green", radius = 2)
    except: 1
    try: out = v.draw_soft_mask(np.asarray(outputs["instances"].to("cpu").pred_masks)[0,:,:], "blue",alpha = 0.3)
    except: 1
    fp = os.path.join("C:/R_projects/deep_learning_playground/detectron2/custom_data2/vid_pred", os.path.basename(fi))
    cv2.imwrite(fp, cv2.cvtColor(out.get_image(), cv2.COLOR_BGR2RGB))
    print(f"{i+1} of {tot}", end = "\r")

In [ ]:
#a = np.asarray(outputs['instances'].pred_keypoint_heatmaps.to("cpu"))[0,1,:,:]
roi_box = outputs['instances'].pred_boxes.tensor
#plt.imshow(a, cmap='hot', interpolation='nearest')
from detectron2.structures.keypoints import Keypoints, heatmaps_to_keypoints

heat_map_og = outputs['instances'].pred_keypoint_heatmaps

print(
    heatmaps_to_keypoints(heat_map_og, roi_box)
)

penalty = -30
nscale = 3
w = heat_map_og.shape[2]
h = heat_map_og.shape[3]
nkps = heat_map_og.shape[1]
grid = torch.tensor([(x, y) for x in range(w) 
              for y in range(h)], 
              device = heat_map_og.device)
r2 = (max([h,w])/nscale)**2

first_kp_pred = heatmaps_to_keypoints2(heat_map_og, roi_box)
first_best_kp_pos = torch.argmax(first_kp_pred[0][:,2])
first_best_kp = first_kp_pred[0,first_best_kp_pos][0:2]

region_mask = grid[
    ((grid[:,0] - int(first_best_kp[0]))**2 + 
    (grid[:,1] - int(first_best_kp[1]))**2) < 
    r2,
    :]

flat_inds = [int(region_mask[i, 0] + region_mask[i, 1] * w) for i in range(region_mask.shape[0])]

for i in range(nkps):
    if(i == first_best_kp_pos):
        continue
    heat_map_og.reshape((3,-1,))[i,[flat_inds]] = torch.tensor(
        penalty,
        device = heat_map_og.device,
        dtype = heat_map_og.dtype
    )

second_kp_pred = heatmaps_to_keypoints2(heat_map_og, roi_box)
second_best_kp_pos = second_kp_pred[0][:,2].topk(2).indices[1]
second_best_kp = second_kp_pred[0,second_best_kp_pos][0:2]

region_mask = grid[
    ((grid[:,0] - int(second_best_kp[0]))**2 + 
    (grid[:,1] - int(second_best_kp[1]))**2) < 
    r2,
    :]

flat_inds = [int(region_mask[i, 0] + region_mask[i, 1] * w) for i in range(region_mask.shape[0])]

for i in range(nkps):
    if(i == first_best_kp_pos or i == second_best_kp_pos):
        continue
    heat_map_og.reshape((3,-1,))[i,[flat_inds]] = torch.tensor(
        penalty,
        device = heat_map_og.device,
        dtype = heat_map_og.dtype
    )

print(
    heatmaps_to_keypoints(heat_map_og, roi_box)
)
fig = plt.figure(figsize=(20,20)) 
fig.add_subplot(2, 2, 1)

plt.imshow(np.asarray(heat_map_og.to("cpu")[0,0,:,:]), cmap='hot', interpolation='nearest')
fig.add_subplot(2, 2, 2)
plt.imshow(np.asarray(heat_map_og.to("cpu")[0,1,:,:]), cmap='hot', interpolation='nearest')
fig.add_subplot(2, 2, 3)
plt.imshow(np.asarray(heat_map_og.to("cpu")[0,2,:,:]), cmap='hot', interpolation='nearest')









In [ ]:
im = cv2.imread(fi)
outputs = predictor(im) 
v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("cat_data_train"), 
                   scale=1, 
               instance_mode=ColorMode.IMAGE_BW
)

final_kpts = heatmaps_to_keypoints_iterative(outputs['instances'].pred_keypoint_heatmaps, roi_box).to("cpu")

try: out = v.draw_circle((final_kpts.tolist())[0][0][:2:], "red", radius = 5)
except: print("Head not detected") 
try: out = v.draw_circle((final_kpts.tolist())[0][1][:2:], "black")
except: print("Middle not detected") 
try: out = v.draw_circle((final_kpts.tolist())[0][2][:2:], "green")
except: print("Tail not detected")

plt.figure(figsize = (10,10))
plt.imshow(out.get_image(), interpolation='nearest')
plt.title(fi)
plt.show()

In [ ]:
from detectron2.structures.keypoints_hack import heatmaps_to_keypoints_iterative

print(heatmaps_to_keypoints_iterative(outputs['instances'].pred_keypoint_heatmaps, roi_box))
print(heatmaps_to_keypoints(outputs['instances'].pred_keypoint_heatmaps, roi_box))

In [ ]:
heatmaps_modify_iterative(outputs["instances"].pred_keypoint_heatmaps, None, 1)

In [ ]:
fig = plt.figure(figsize=(20,20)) 
fig.add_subplot(2, 2, 1)
plt.imshow(np.asarray(outputs["instances"].pred_keypoint_heatmaps.to("cpu")[0,0,:,:]), cmap='hot', interpolation='nearest')
fig.add_subplot(2, 2, 2)
plt.imshow(np.asarray(outputs["instances"].pred_keypoint_heatmaps.to("cpu")[0,1,:,:]), cmap='hot', interpolation='nearest')
fig.add_subplot(2, 2, 3)
plt.imshow(np.asarray(outputs["instances"].pred_keypoint_heatmaps.to("cpu")[0,2,:,:]), cmap='hot', interpolation='nearest')

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join("./output4/", "model_0005999.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9  # set a custom testing threshold
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("cat_data_test",tasks=["bbox","segm","keypoints"], distributed=False, max_dets_per_image=1, 
                          use_fast_impl=False, allow_cached_coco= False, output_dir="./output/", kpt_oks_sigmas = [0.1,0.5,0.1])
val_loader = build_detection_test_loader(cfg, "cat_data_test")
detectron2.evaluation.inference_on_dataset(predictor.model, val_loader, evaluator)

In [ ]:
nuts_metadata = MetadataCatalog.get('cat_data_test')
dataset_dicts = DatasetCatalog.get("cat_data_test")
fig = plt.figure(figsize=(10,30)) 

i = 0
for d in dataset_dicts[0:10]:
    i += 1
    fig.add_subplot(10, 2, i) 
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1], metadata=nuts_metadata , scale=0.5)
    try: out = v.draw_circle((outputs["instances"].pred_keypoints.tolist())[0][0][:2:], "red", radius = 5)
    except: print("Head not detected\r") 
    try: out = v.draw_circle((outputs["instances"].pred_keypoints.tolist())[0][1][:2:], "blue")
    except: print("Middle not detected\r") 
    try: out = v.draw_circle((outputs["instances"].pred_keypoints.tolist())[0][2][:2:], "green")
    except: print("Tail not detected\r") 
    #out = v.draw_dataset_dict(d)
    plt.imshow(out.get_image()[:, :, ::-1])
    


In [ ]:
fig = plt.figure(figsize=(20,100)) 

i = 0
for d in dataset_dicts:
    i += 1
    fig.add_subplot(10, 2, i) 
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    visualizer = Visualizer(img[:, :, ::-1], metadata=nuts_metadata , scale= 0.3, instance_mode= ColorMode.IMAGE_BW)
    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    out = visualizer.draw_dataset_dict(d)
    plt.title(d["file_name"])
    plt.imshow(out.get_image()[:, :, ::-1])
